In [ ]:
%matplotlib inline
import re
from datetime import datetime
import numpy as np
from mpmath import mp
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import math
import warnings
from ipywidgets import interact, interactive, fixed, interact_manual
plt.rcParams['figure.dpi'] = 90
plt.rcParams['figure.figsize'] = [24.0, 16.0]
plt.rcParams['text.latex.unicode'] = False
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.fontset'] = 'stixsans'
plt.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
def events(log):
    EVENT_COST = 1954
    start = None
    discount = 0
    events = []
    for line in log.split('\n'):
        match = re.search(r'^(ENTER|LEAVE) (\w+) (\d+) (\d+)$', line)
        if match:
            (event, name, gas, mem) = match.groups()
            gas = -int(gas)
            mem = int(mem)
            if start:
                gas -= start
            else:
                start = gas
                gas = 0
            begin = event == 'ENTER'
            if not name in ['calldata', 'transaction']:
                discount += EVENT_COST
            gas -= discount
            events += [(gas, begin, name, mem)]
    return events

In [ ]:
def process(events):
    stack = []
    spans = []
    histogram = {}
    last = None
    for event in events:
        (time, begin, name, _) = event
        # Collect top of stack histogram
        if last:
            top_of_stack = stack[-1][2]
            duration = time - last
            if top_of_stack in histogram:
                histogram[top_of_stack] += duration
            else:
                histogram[top_of_stack] = duration
        last = time
        # Compute spans
        if begin:
            stack += [event]
        else:
            (start, _, previous_name, _) = stack[-1]
            stack = stack[:-1]
            assert name == previous_name
            spans += [(len(stack), start, time, name)]
    histogram = {k: v for k, v in sorted(histogram.items(), key=lambda item: -item[1])}
    print(len(spans), len(histogram))
    return spans, histogram

In [ ]:
97853368 - 96324406

In [ ]:
log = """
ENTER transaction 100000000 0
ENTER calldata 100000000 0
LEAVE calldata 98950632 0
ENTER verify_proof 98703813 81184
ENTER constraint_calculations 98533290 116009
LEAVE constraint_calculations 97882702 117417
ENTER check_commitments 97880694 117417
ENTER verify_merkle_proof 97850843 120777
LEAVE verify_merkle_proof 97311142 121225
ENTER verify_merkle_proof 97281680 123241
LEAVE verify_merkle_proof 96741979 123689
LEAVE check_commitments 96739960 123689
ENTER fri_check 96737941 123689
LEAVE fri_check 86039681 152041
LEAVE verify_proof 86036420 152041
LEAVE transaction 86034373 152041
"""

In [ ]:
e = events(log)

In [ ]:
e[-1]

In [ ]:
flame, pie = process(e)

In [ ]:
pie

In [ ]:
x = np.array([e[0] for e in e])
y = np.array([e[-1] for e in e])

In [ ]:
def flamegraph(spans):
    labels = list(set([span[3] for span in spans]))
    maxt = spans[-1][2]
    colour_map = mpl.cm.rainbow(np.linspace(0, 1, len(labels)))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim((0,maxt))
    ax.set_ylim((0,7))

    for span in spans:
        (depth, start, end, label) = span
        colour = colour_map[labels.index(label)]
        rectangle = mpl.patches.Rectangle((start, depth), end - start, 1, fc=colour, ec='white')
        ax.add_patch(rectangle)
        if end - start > 0.01 * maxt:
            ax.text(start + 0.005 * maxt, depth+0.05, label)

In [ ]:
flamegraph(flame)
plt.plot(x, 6.5 * y / max(y), color='black')

In [ ]:
labels = list(data[-1][1].keys())
values = np.array([[hist[label] for label in labels] for (_, hist) in data]).T
fig, ax = plt.subplots()
y = values / np.sum(values, axis=0)
ax.stackplot(np.log2(x), y[::-1, :], labels=labels[::-1])
ax.set_xlim((10,15))
ax.set_ylim((0,1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')

plt.show()